In [1]:
#import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sts
import seaborn as sns

%matplotlib inline

#import conviction files
#from conviction_helpers import *
#from conviction_system_logic3 import *
from bonding_curve_eq import *

System initialization

In [2]:
hatch_raise = 100000 # fiat units
hatch_price = .1 #fiat per tokens
theta = .50 #share of funds going to funding pool at launch

R0 = hatch_raise*(1-theta)
F0 = hatch_raise*theta
S0 = hatch_raise/hatch_price

kappa = 2
V0 = invariant(R0,S0,kappa)
P0 = spot_price(R0, V0, kappa)

agent initialization

In [3]:
#number of agents
n= 100

#gain factors
g = np.random.normal(1, .05, size=n)
phat0 = g*F0/S0

#holdings fiat
h = sts.expon.rvs( loc=10,scale=10, size=n)

#holdings tokens
s_dist = sts.expon.rvs(loc=10, scale=10, size=n)
s0 = s_dist/sum(s_dist)*S0

#lambda for revenue process
lam = 200

#phi for exiting funds
phi = .05

#beta is param for armijo rule
beta = .9

In [4]:
params= {
    'kappa': [kappa],
    'lambda': [lam],
    'gains': [g],
    'population':[n],
    'beta':[beta],
    'phi': [phi],
    'invariant': [V0]}

In [5]:
initial_conditions = {'holdings': h,
                      'tokens': s0,
                      'supply': S0,
                      'prices': phat0,
                      'funds':F0,
                      'reserve': R0,
                      'spot_price': P0,
                      'action': {}}

In [6]:
initial_conditions

{'action': {},
 'funds': 50000.0,
 'holdings': array([11.44967426, 14.27340299, 11.34460577, 16.83844489, 11.25574143,
        38.43594327, 11.42707407, 19.78337576, 18.92621004, 13.76349783,
        20.56410232, 46.70590673, 11.39419877, 16.36850516, 23.93412804,
        13.16706921, 17.47432165, 24.08451343, 10.31676   , 22.76223411,
        26.52489583, 23.50714218, 10.00885272, 36.39730482, 10.66712987,
        20.227257  , 43.40147235, 15.96228714, 12.23014236, 24.86208805,
        15.15989215, 34.42215238, 15.71773678, 40.79049265, 34.35363414,
        18.53725115, 28.23462521, 12.53490108, 38.00409625, 27.43551221,
        18.01672824, 19.11400584, 11.6431133 , 10.30271338, 10.64391881,
        10.64852586, 30.93014093, 19.38071005, 22.30292307, 11.68379312,
        39.84043761, 24.574661  , 33.22214478, 13.69030971, 12.00214066,
        12.72863488, 21.56760105, 13.74530629, 10.39138789, 14.01340641,
        17.00097386, 15.41768633, 13.31798796, 10.77507215, 24.24516956,
     

In [7]:
#change in F (revenue and spending accounted for)
def revenue_process(params, step, sL, s):
    lam = params['lambda']
    rv = sts.expon.rvs(loc = .001, scale=1/lam)
    delF= 1-1/lam+rv
    
    #avoid the crash (temporary hacks, tune martingale process better)
    if delF <1:
        if s['funds'] <1000:
            delF =100
    
    return({'delF':delF})

In [8]:
def update_funds(params, step, sL, s, _input):
    
    funds = s['funds']*_input['delF']
    
    key = 'funds'
    value = funds
    
    return (key, value)

In [9]:
def update_prices(params, step, sL, s, _input):
    
    g = params['gains']
    phat = g*s['funds']/s['supply']
    
    key = 'prices'
    value = phat
    
    return (key, value)

In [10]:
#change in F (revenue and spending accounted for)
def choose_agent(params, step, sL, s):
    n = params['population']
    rv = np.random.randint(0,n)
    return({'agent':rv})

In [11]:
def agent_action(params, step, sL, s, _input):
    
    a = _input['agent']
    h_a = s['holdings'][a]
    phat_a = s['prices'][a]
    s_a =  s['tokens'][a]
    p = s['spot_price']
    
    beta = params['beta']
    
    if p>phat_a: #equiv: pbar(0)>phat_a
        mech = 'burn'
        #approx for burn s.t. p=phat
        #armijo style
        amt = s_a
        
        def pbar(amt):
            output = withdraw_with_tax(amt, s['reserve'],s['supply'], params['invariant'], params['phi'], params['kappa'])
            
            if not(output[2])>0:
                return np.Infinity
            else:
                return output[2]
        
        
        while pbar(amt)< phat_a:
            amt = amt*beta
        
    else: # p<phat_a; #equiv pbar(0)<phat_a
        mech = 'bond'
        #approx for buy s.t. p=phat
        #armijo style
        amt = h_a
        
        def pbar(amt):
            output = mint(amt, s['reserve'],s['supply'], params['invariant'], params['kappa'])
            
            if not(output[1])>0:
                return 0
            else:
                return output[1]
        
        while pbar(amt)> phat_a:
            amt = amt*beta
    
    #print(mech)
    #print(amt)
    #print(pbar(amt))
    key = 'action'
    value = {'agent':a, 'mech':mech, 'amt':amt, 'pbar':pbar(amt)}
    
    return (key, value)

In [12]:
def resolve_action(params, step, sL, s):
    action = s['action']
    a = action['agent']
    amt = action['amt']
    h_a = s['holdings'][a]
    s_a =  s['tokens'][a]
    R = s['reserve']
    S = s['supply']
    F = s['funds']
    V0 = params['invariant']
    
    if action['mech'] == 'bond':
        h_a = h_a-amt
        dS, pbar = mint(amt, R,S, V0, params['kappa'])
        R = R+amt
        S = S+dS
        s_a = s_a+dS
        P = spot_price(R, V0, kappa)
    
    elif action['mech'] == 'burn':
        s_a = s_a-amt
        dR, pbar = withdraw(amt, R,S, V0, params['kappa'])
        R = R-dR
        F = F + params['phi']*dR
        S = S-amt
        h_a = h_a + (1-params['phi'])*dR
        P = spot_price(R, V0, kappa)
    
    return({'F':F, 'S':S, 'R':R,'P':P, 'a':a,'s_a':s_a, 'h_a':h_a})

In [13]:
def update_F(params, step, sL, s, _input):
    
    F = _input['F']
    
    key = 'funds'
    value = F
    
    return (key, value)

def update_S(params, step, sL, s, _input):
    
    S = _input['S']
    
    key = 'supply'
    value = S
    
    return (key, value)

def update_R(params, step, sL, s, _input):
    
    R = _input['R']
    
    key = 'reserve'
    value = R
    
    return (key, value)

def update_P(params, step, sL, s, _input):
    
    P = _input['P']
    
    key = 'spot_price'
    value = P
    
    return (key, value)

def update_holdings(params, step, sL, s, _input):
    
    h_a = _input['h_a']
    a = _input['a']
    
    h = s['holdings']
    h[a] = h_a
    
    key = 'holdings'
    value = h
    
    return (key, value)

def update_tokens(params, step, sL, s, _input):
    
    s_a = _input['s_a']
    a = _input['a']
    
    tokens = s['tokens']
    tokens[a] = s_a
    
    key = 'tokens'
    value = tokens
    
    return (key, value)

In [14]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# The Partial State Update Blocks
partial_state_update_blocks = [
    { 
        'policies': { 
            #new proposals or new participants
            'random': revenue_process
        },
        'variables': {
            'funds': update_funds,
            'prices': update_prices
        }
    },
    {
      'policies': {
          'random': choose_agent
        },
        'variables': { 
            'action': agent_action, 
        }
    },
    {
      'policies': {
          'act': resolve_action,
        },
        'variables': {
            'funds': update_F, #
            'supply': update_S, 
            'reserve': update_R,
            'spot_price': update_P,
            'holdings': update_holdings,
            'tokens': update_tokens
        }
    }
]

In [15]:
time_periods_per_run = 300
monte_carlo_runs = 1

from cadCAD.configuration.utils import config_sim
simulation_parameters = config_sim({
    'T': range(time_periods_per_run),
    'N': monte_carlo_runs,
    'M': params
})

In [16]:
from cadCAD.configuration import append_configs
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# The configurations above are then packaged into a `Configuration` object
append_configs(
    initial_state=initial_conditions, #dict containing variable names and initial values
    partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
    sim_configs=simulation_parameters #dict containing simulation parameters
)

[{'N': 1, 'T': range(0, 300), 'M': {'kappa': 2, 'lambda': 200, 'gains': array([1.01535303, 0.94669471, 0.99863361, 0.91696039, 0.97030072,
       0.9221604 , 0.87084168, 1.04207957, 0.96289949, 0.91718973,
       0.99693053, 1.02431411, 0.92674754, 0.90909958, 1.03321212,
       0.96622344, 0.9994914 , 1.01483049, 1.04987409, 0.98681474,
       0.97199609, 0.93780697, 1.04419426, 0.94070826, 0.890677  ,
       0.97459264, 1.01360303, 0.94063754, 1.02463315, 1.01350909,
       0.8921601 , 1.01607629, 0.99305991, 1.00907995, 1.01759725,
       1.01668989, 0.92590389, 1.01565762, 0.99027221, 0.97351841,
       1.00706862, 1.01969654, 0.96667426, 0.98849401, 0.95999794,
       1.06593137, 0.96815824, 0.93645407, 0.97590572, 1.05963953,
       1.06745931, 1.046569  , 0.99893183, 0.99527535, 0.99574013,
       1.01067462, 0.97592448, 0.99275056, 1.0387044 , 1.00212574,
       1.02541185, 1.07054733, 0.89455423, 1.03301249, 1.00769868,
       1.0597725 , 0.87798826, 0.97200691, 1.06589903, 0.

In [17]:
from tabulate import tabulate
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
from cadCAD import configs
import pandas as pd

exec_mode = ExecutionMode()
multi_proc_ctx = ExecutionContext(context=exec_mode.multi_proc)
run = Executor(exec_context=multi_proc_ctx, configs=configs)

In [18]:
i = 0
verbose = False
results = {}
for raw_result, tensor_field in run.execute():
    result = pd.DataFrame(raw_result)
    if verbose:
        print()
        print(f"Tensor Field: {type(tensor_field)}")
        print(tabulate(tensor_field, headers='keys', tablefmt='psql'))
        print(f"Output: {type(result)}")
        print(tabulate(result, headers='keys', tablefmt='psql'))
        print()
    results[i] = {}
    results[i]['result'] = result
    results[i]['simulation_parameters'] = simulation_parameters[i]
    i += 1
    


                            __________   ____ 
          ________ __ _____/ ____/   |  / __ \
         / ___/ __` / __  / /   / /| | / / / /
        / /__/ /_/ / /_/ / /___/ ___ |/ /_/ / 
        \___/\__,_/\__,_/\____/_/  |_/_____/  
        by BlockScience
        
Execution Mode: multi_proc: [<cadCAD.configuration.Configuration object at 0x1a1d44d048>]
Configurations: [<cadCAD.configuration.Configuration object at 0x1a1d44d048>]


/Users/zargham/Documents/GitHub/conviction/bonding_curve_eq.py:56: RuntimeWarning: invalid value encountered in double_scalars
  realized_price = quantity_recieved/deltaS
/Users/zargham/Documents/GitHub/conviction/bonding_curve_eq.py:47: RuntimeWarning: invalid value encountered in double_scalars
  realized_price = deltaR/deltaS
/Users/zargham/Documents/GitHub/conviction/bonding_curve_eq.py:56: RuntimeWarning: divide by zero encountered in double_scalars
  realized_price = quantity_recieved/deltaS
/Users/zargham/Documents/GitHub/conviction/bonding_curve_eq.py:47: RuntimeWarning: divide by zero encountered in double_scalars
  realized_price = deltaR/deltaS
/Users/zargham/Documents/GitHub/conviction/bonding_curve_eq.py:37: RuntimeWarning: invalid value encountered in double_scalars
  realized_price = deltaR/deltaS
/Users/zargham/Documents/GitHub/conviction/bonding_curve_eq.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  realized_price = deltaR/deltaS
Process ForkPoo

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/multiprocess/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.6/site-packages/multiprocess/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.6/site-packages/multiprocess/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/anaconda3/lib/python3.6/site-packages/multiprocess/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/anaconda3/lib/python3.6/site-packages/pathos/helpers/mp_helper.py", line 15, in <lambda>
    func = lambda args: f(*args)
  File "/anaconda3/lib/python3.6/site-packages/cadCAD/engine/__init__.py", line 47, in <lambda>
    results = p.map(lambda t: t[0](t[1], t[2], t[3], t[4], t[5], t[6]), l)
  File "/anaconda3/lib/python3.6/site-packages/cadCAD/engine/simulation.py", line 229, in simulation
    list(range(runs))
  File "/anaconda3/lib/python3.6/site-packag

In [ ]:
experiment_index = 0
df = results[experiment_index]['result']

In [ ]:
df.funds.plot()

In [ ]:
(df.funds.diff()/df.funds).hist()

In [ ]:
rdf = df[df.substep == 3].copy()

In [ ]:
rdf['token_wts'] = (rdf.tokens/rdf.supply)
rdf['wt_mean_price'] = (rdf.token_wts*rdf.prices).apply(sum)

In [ ]:
rdf.prices.apply(np.min).plot()
rdf.prices.apply(np.median).plot()
rdf.prices.apply(np.mean).plot()
rdf.wt_mean_price.plot()
rdf.prices.apply(np.max).plot()
rdf.spot_price.plot()
plt.legend(['min', 'median','mean','wt mean','max', 'spot'])

In [ ]:
rdf.prices.apply(np.min).apply(np.log).plot()
rdf.prices.apply(np.median).apply(np.log).plot()
rdf.prices.apply(np.mean).apply(np.log).plot()
rdf.wt_mean_price.apply(np.log).plot()
rdf.prices.apply(np.max).apply(np.log).plot()
rdf.spot_price.apply(np.log).plot()
plt.legend(['min', 'median','mean','wt mean','max', 'spot'])

In [ ]:
rdf['median_price']=rdf.prices.apply(np.median)
rdf['mean_price']=rdf.prices.apply(np.mean)

In [ ]:
(np.sign(rdf['mean_price']-rdf['spot_price'])*(rdf['mean_price']-rdf['spot_price'])**2).apply(np.log10).plot(alpha=1)
(-np.sign(rdf['mean_price']-rdf['spot_price'])*(rdf['mean_price']-rdf['spot_price'])**2).apply(np.log10).plot(alpha=.5)
plt.legend(['over','under'])

In [ ]:
rdf['est_err'] = rdf.spot_price - rdf.wt_mean_price
rdf['sq_est_err'] = rdf['est_err']**2

In [ ]:
rdf.est_err.plot()

In [ ]:
rdf.est_err.plot(logy=True)

In [ ]:
#tail T
T = 50
rdf.tail(T).prices.apply(np.min).plot()
rdf.tail(T).prices.apply(np.median).plot()
rdf.tail(T).prices.apply(np.mean).plot()
rdf.tail(T).wt_mean_price.plot()
rdf.tail(T).prices.apply(np.max).plot()
rdf.tail(T).spot_price.plot()
plt.legend(['min', 'median','mean','wt mean','max', 'spot'])

In [ ]:
bond_amts = [rdf.iloc[k].action['amt'] for k in range(time_periods_per_run) if rdf.iloc[k].action['mech']=='bond']
burn_amts = [rdf.iloc[k].action['amt'] for k in range(time_periods_per_run) if rdf.iloc[k].action['mech']=='burn']

In [ ]:
plt.hist(bond_amts)

In [ ]:
plt.hist(burn_amts)

In [ ]:
rdf['invariant'] = rdf.supply.apply(lambda x: x**kappa)/rdf.reserve

In [ ]:
rdf['resid'] = rdf.invariant-V0

In [ ]:
rdf.resid.apply(np.log10).plot()

In [ ]:
rdf.plot(x='reserve', y='supply', kind='scatter', alpha=.5)
axis = plt.axis()
xrange = np.arange(axis[0], axis[1], (axis[1]-axis[0])/100)
yrange = np.array([supply(x, V0, kappa) for x in xrange ])
plt.plot(xrange, yrange, 'y')
plt.title('Bonding Curve Invariant')
plt.legend(['Invariant', 'Observed Data'])

In [ ]:
(rdf.tokens.apply(sum)-rdf.supply).plot()

In [ ]:
def gini(x):

    # Mean absolute difference
    mad = np.abs(np.subtract.outer(x, x)).mean()
    # Relative mean absolute difference
    rmad = mad/np.mean(x)
    # Gini coefficient
    g = 0.5 * rmad
    return g

In [ ]:
rdf['gini_h'] = rdf.holdings.apply(gini)

In [ ]:
rdf.gini_h.plot()

In [ ]:
rdf['gini_s'] = rdf.tokens.apply(gini)

In [ ]:
rdf.gini_s.plot()

In [ ]:
rdf.tokens.apply(np.count_nonzero).plot()

In [ ]:
rdf['asset_value'] = rdf.holdings + rdf.spot_price*rdf.tokens

In [ ]:
rdf['gini'] = rdf.asset_value.apply(gini)

In [ ]:
rdf.gini.plot()

In [ ]:
rdf.tokens.sum()